## Övning 6
__John Landeholt__

johnlan@kth.se


__agenda:__

* hammingavstånd
* komprimering
* kryptering

__Hammingavstånd__

Är inte en komprimeringsalgoritm, utan ett sätt att avgöra mellan två strängar av samma storlek det minsta antalet substitutioner som krävs för att omvandla $s_1$ till $s_2$. Detta kallas även första minsta felkorrigeringen möjligt mellan två strängar.

In [28]:
def hamming_distance(s1,s2):
    return sum(xi != yi for xi, yi in zip(s1, s2))

hamming_distance('Tilda', 'Milda')

1

__Tentafråga 2019-03-13 [6] E__

Givet en teckentabell med deras decimaltal och motsvarande binärtal

    A 65 1000001
    B 66 1000010
    C 67 1000011
    ...
    a 97 1100001
    b 98 1100010
    c 99 1100011
    ...

Fråga. Vad är minsta hammingavståndet för de __binära koderna__ i hela teckentabellen

__Lösningsförslag__

Det är enkelt att se att __minsta__ avståndet måste vara __1__. A -> B till exempel.


### Komprimering

Det finns två sorters komprimering:

* non-lossy compression - konverterar data
* lossy compression - tar bort data

__Följdlängdskodning__

Är en av de mer intuitiva kompressionsalgoritmerna, som minimerar följder av samma tecken genom att eliminera alla dubbletter och istället spara antalet som en siffra.

Men är inte så värst användbar i verkligenheten, då texter inte brukar innehålla långa följder av samma tecken.

In [1]:
from sys import getsizeof

def comp_rate(text,encoding):
    before = getsizeof(text)
    after = getsizeof(encoding)
    return (before - after) / after * 100

def rle_encoding(text):
    encoding = ''
    prev = ''
    count = 1
    for c in text:
        if c != prev:
            if prev:
                if count == 1:
                    encoding += prev
                else:
                    encoding += str(count) + prev
            count = 1
            prev = c
        else:
            count += 1
    encoding += str(count) + prev
    
    
    rate = comp_rate(text,encoding)
    print(f'compressed input with {round(rate, 2)}%')
    return encoding

encoding = rle_encoding('ÅÅÅÅH! JAAAAAAA! AAAAAAAAAAAAH.')
print(encoding)

In [2]:
def rle_decoding(encoding):
    decoding = ''
    count = ''
    for c in encoding:
        if c.isnumeric():
            count += c
        else:
            if count == '':
                decoding += c
            else:
                decoding += int(count) * c
                count = ''
    return decoding
text = rle_decoding(encoding)
print(text)

__Huffmankodning__

<img src="img/huffman.png" style="float:right" />

Går ut på att __räkna__ hur vanliga tecken är i en text. Där det tecknet med högst frekvens får kortast binärkod.

Man börjar med att räkna frekvensen av alla tecken i texten, sedan placerar du det i en prioritetskö, där den med högst prioritet har minst sannolikhet (en min-heap).

Sedan för varje par i kön så samlar du alla __0__ för det vänstra paret och __1__ för det högra paret, tills det endast finns ett par i kön, vilket kommer att vara roten som har sannolikheten __1__.

Detta innebär att du alltså börjar med löven i trädet och bygger dig upp för varje par.

På bilden ser vi att paret __(R,G)__ summeras upp till __2__ för att sedan i nästa loop bli till paret __(I, (R,G))__ vars summa är __4__ osv..

Sedan traversar man trädet igen och plockar med sig kodningen, som för __G__ blir __0111__

> __notera__: huffmankodning är endast non-lossy om `frekvenstabellen` sparas. Annars är den lossy.

In [2]:
from shared import Min_heap, Huffman_node
from collections import defaultdict
from time import sleep

def print_heap(heap):
    for n in heap.array[1:]:
        print(n, end= ' ')
    print()
        
def huffman_encode(text):
    freq = defaultdict(int)
    heap = Min_heap()
    
    for c in text: freq[c] += 1
    for k,v in freq.items(): heap.insert(Huffman_node(k,v))
    while len(heap) > 1:
        print_heap(heap)
        left, right = heap.pop(), heap.pop()
        for p in left.pairs: p += '0'
        for p in right.pairs: p += '1'
        n = Huffman_node.merge(left, right)
        heap.insert(n)
        sleep(3)
    print_heap(heap)
    encoding = ''
    node = heap.pop()
    for c in text:
        code = node[c]
        if code:
            encoding += code + ' '
    return encoding[:-1], freq

encoding, freq = huffman_encode('HAHA!IIIIIIH!AHRG...')
print(encoding)

{G: None}: 1 {!: None}: 2 {R: None}: 1 {I: None}: 6 {H: None}: 4 {A: None}: 3 {.: None}: 3 
{G: 0, R: 1}: 2 {.: None}: 3 {!: None}: 2 {I: None}: 6 {H: None}: 4 {A: None}: 3 
{A: None}: 3 {.: None}: 3 {H: None}: 4 {I: None}: 6 {G: 00, R: 01, !: 1}: 4 
{H: None}: 4 {G: 00, R: 01, !: 1}: 4 {I: None}: 6 {A: 0, .: 1}: 6 
{A: 0, .: 1}: 6 {I: None}: 6 {H: 0, G: 100, R: 101, !: 11}: 8 
{H: 0, G: 100, R: 101, !: 11}: 8 {A: 00, .: 01, I: 1}: 12 
{H: 00, G: 0100, R: 0101, !: 011, A: 100, .: 101, I: 11}: 20 
00 100 00 100 011 11 11 11 11 11 11 00 011 100 00 0101 0100 101 101 101


In [3]:
from shared import huffman_decode
huffman_decode(encoding, freq)

'HAHA!IIIIIIH!AHRG...'

### Huffmankodning

__Hur gör vi det för hand?__

Vi prövar med stringen "man är mans gamman".

Steg:

1. Beräkna frekvensen för alla tecken
2. Sortera så att det tecknet med minst frekvens är först
3. Placera dem i en kö så att det är minst -> störst
4. poppa kön 2 gånger, summera frekvenserna för paret ge första tecknet en __0__ och andra en __1__
5. pusha paret in i kön igen
6. repetera punkt 4-5 tills det endast finns en nod i kön.

[länk till hela processen](http://www.umsl.edu/~siegelj/information_theory/codes/huffman.html)

__Lempel-Ziv__

<img src="https://media.emailonacid.com/wp-content/uploads/2019/03/2019-GifsInEmail.gif" style="float:right" />

Går ut på att man bygger upp strängar i en tabell, sådan att det finns en kod för varje tecken och påbyggnader av de redan existerande tecknen i tabellen.

Lempel-Ziv är en av de mest användna `komprimeringsalgoritmerna` som används i Linux (unix), gifs, zip m.m. Likt tidigare algoritmer, så är denna också `non-lossy`.

In [27]:
def lzw_encode(data):
    table = {}
    encoding = ''
    s = ''
    for c in data:
        _next = s + str(c)
        if _next in table:
            s = _next
        else:
            try:
                code = str(table[s])
            except:
                code = ''
            finally:
                encoding += code + c
            
            table[_next] = len(table)
            s = ''
    if s in table:
        encoding += str(table[s])
    return encoding, table

encoding, table = lzw_encode('NÄSSNUVSNORSNOK')
print(encoding)

NÄS2NUV3OR6K


### Kryptering

Det finns en hel rös av olika krypteringsalgoritmer att lära sig. Här är en rad nyttiga exempel:

* Transpositionschiffer
* Chiffer
    * rot13
    * Bokchiffer
    * One-time pad
* Asymmetrisk kryptering
* RSA



__Transpositionschiffer__

Går ut på att tranformera en text i en specifik ordning. I vardagen brukar man göra det genom en matris. Där man delar upp texten på rader och sedan väljer en sekvens eller ordning som man väljer en bokstav per rad.

In [49]:
import math

def transpose_chifer(text):
    M = len(text)
    N = 3
    matrix = []
    K = math.ceil(M / N)
    i = 0
    chifer = ''
    for j in range(N):
        matrix.append(list(text[i:K*(j+1)]))
        if len(matrix[j]) != K:
            matrix[j] += ['_'] * abs(K - len(matrix[j]))
        i += K
    if K*N < M:
        if len(text[i:]) < K:
            s = text[i:] + '_' * abs(K - len(text[i:]))
        else:
            s = text[i:]
        matrix.append(s)
    for i in range(K):
        for row in matrix:
            chifer += row[i]
    return chifer
        
transpose_chifer('JOHN LANDEHOLT')

'JLHOAOHNLNDT E_'

__ROT13 chiffer__

Går ut på att man har mappat om alfabetet så att A blir till N, genom att förskjuta alfabetet 13 bokstäver.

In [73]:
from string import ascii_lowercase
def rot13(text):
    char2num = {c:i for i,c in enumerate(ascii_lowercase)}
    num2char = {i:c for i,c in enumerate(ascii_lowercase)}
    chifer = ''
    for c in text.lower():
        try:
            chifer += num2char[(char2num[c] + 13) % len(num2char)]
        except:
            pass
    return chifer

rot13('John Landeholt')

'wbuaynaqrubyg'

__bokchiffer__

Går ut på att man väljer en bok, gärna en stor bok, så att man kan enkoda rika texter. Bokchiffer går ut på att parterna har en kopia var av samma upplaga av någon bok och sedan skapar de sifforemsor, där första siffran indikerar vilken sida och andra vilket ord.

    There was a man that was hiding issues from his family. 
    He was bankrupt and in financial ruins after his business had failed.
    He couldn't even afford plain white bread to the family, but the family was
    not allowed to see the sight of this, so he hatched a plan to make it all back.

Om vi tar denna påhittade text som exempel där varje rad representerar en sida.

Vad blir då sifferremsan `1 7 2 5 3 5 4 6`?

__Lösningsförslag__

There was a man that was __hiding__ issues from his family.

He was bankrupt and __in__ financial ruins after his business had failed.

He couldn't even afford __plain__ white bread to the family, but the family was

not allowed to see the __sight__ of this, so he hatched a plan to make it all back.